In [1]:
import MySQLdb
import node_edge_gen as neg
import networkx as nx

from geopy.distance import great_circle


In [99]:
# DEFINE SQL LOG IN 
HOST = 'localhost'
USER = 'root'
PASSWD = ''
DATABASE = 'bk_map'


In [100]:
#CONNECT TO MySQL
db_connect = MySQLdb.connect(
    host = HOST,
    user = USER,
    passwd = PASSWD,
    db = DATABASE)

cur = db_connect.cursor()

In [4]:
def nearest_node(lat_input, lon_input):
    dis_near=0.05
    find_nearby_nodes = ('''SELECT nodeid, latitude, longitude FROM bk_nodes 
                WHERE latitude BETWEEN %s AND %s
                AND longitude BETWEEN %s AND %s
                ''')
    node_range = (lat_input-dis_near, lat_input+dis_near, lon_input-dis_near, lon_input+dis_near)

    cur.execute(find_nearby_nodes,node_range)  
    rows = cur.fetchall()
    if len(rows) == 0:
        return 'NULL'
    #for row in rows:
    else:
        dist = [rows[0][0],great_circle((lat_input,lon_input),rows[0][1:3]).miles ]
        for row in rows[1:]:
            node_dist =  great_circle((lat_input,lon_input),row[1:3]).miles
            if dist[1] > node_dist:
                dist = [row[0],node_dist]
        return dist[0]

def distance_btw_nodes(node1,node2,node_dic):
    return great_circle(node_dic[node1], node_dic[node2]).miles

def accident_weight_btw_nodes(node1, node2, weight_dic):
##    node1 = str(node1)
##    node2 = str(node2)
    if node1 in weight_dic and node2 in weight_dic:
        return weight_dic[node1] + weight_dic[node2]
    elif node1 in weight_dic and node2 not in weight_dic:
        return weight_dic[node1]
    elif node1 not in weight_dic and node2 in weight_dic:
        return weight_dic[node2]
    else:
        return 0

In [89]:
#SQL graph into nx #MAKE THIS SMALLER BASED ON THE INPUT RANGE
cur.execute('''SELECT nodeid, latitude, longitude FROM bk_nodes
                ''')  
nodes_sql = cur.fetchall()

cur.execute('''SELECT edgeid, node1, node2 FROM bk_edges
                ''')  
edges_sql = cur.fetchall()

cur.execute("""SELECT id, latitude, longitude , n_ped_inj  
    FROM bk_trafficaccidents 
    WHERE n_ped_inj > 0
    AND date BETWEEN '2014-09-01' AND '2015-09-01'
    """)
accidents_sql = cur.fetchall()



In [6]:
node_dic = {}
for node in nodes_sql:
    node_dic[node[0]] = node[1:] 

edge_dic = {}
for edge in edges_sql:
    edge_dic[edge[0]]= edge[1:]
    
acc_dic = {}
for a in range(0,len(accidents_sql)):
    node = nearest_node(accidents_sql[a][1],accidents_sql[a][2])
    acc_dic[accidents_sql[a][0]] = accidents_sql[a][1],accidents_sql[a][2],accidents_sql[a][3], node


In [7]:
def gen_network(node_dic, edge_dic):
    G = nx.Graph()
    for d in node_dic:
        G.add_node(d)
    for e in edge_dic:
        G.add_edge(edge_dic[e][0],edge_dic[e][1])

    print 'NODES: ', G.number_of_nodes()
    print 'EDGES: ', G.number_of_edges()
    return G

In [8]:
G = gen_network(node_dic,edge_dic)

NODES:  2473
EDGES:  3922


In [10]:
# adding the attributes of the neighbor list to each node in the network
neighbor_dic = {}
for node in G.nodes():
    neighbor_dic[node] = G.neighbors(node)
    
nx.set_node_attributes(G, 'neighbors', neighbor_dic)

In [23]:
node_neigh = nx.get_node_attributes(G,'neighbors')
tmp_dic = {}
feature_dict = acc_dic 
for feat in feature_dict:
    node = feature_dict[feat][3]
    if node != 'NULL':
        if node in tmp_dic:
            tmp_dic[node] =  tmp_dic[node] + 1
        else:
            tmp_dic[node] = feature_dict[feat][2]
                
    else:
        pass


In [36]:
#for node in node_neigh:
#    for no in node_neigh[node]:
#        print tmp_dic[no]

print len(tmp_dic)
print len(node_dic)

traffic_dic = {}
for node in node_dic:
    if node in tmp_dic:
        traffic_dic[node] = float(tmp_dic[node])
    else:
        traffic_dic[node] = 0

#print traffic_dic
nx.set_node_attributes(G, 'traffic', traffic_dic)

660
2473


In [86]:
def make_nearestneighborhood_weights(node_dict,edge_dict,feature_dict,feature_factor):
    traffic_weightnn = {}
    for node in neighbor_dic:
        traffic_weight = 0
        for no in neighbor_dic[node]:
            traffic_weight += traffic_dic[no] 
        #weight = traffic_weight + traffic_dic[node]    
        if traffic_weight + traffic_dic[node] != 0:
             weight = traffic_dic[node] / (traffic_weight + traffic_dic[node])
        else:
            weight = 0
        traffic_weightnn[node] = weight
    tmp_edge = {}
    
    for edge in edge_dic:
        #print edge_dict[edge][0], edge_dict[edge][1]
        acc_wei = accident_weight_btw_nodes(edge_dic[edge][0], edge_dic[edge][1], traffic_weightnn)
        weight = ((feature_factor*acc_wei) + 1)*distance_btw_nodes(edge_dic[edge][0],edge_dic[edge][1],node_dic)
        tmp_edge[edge] = edge_dic[edge][0], edge_dic[edge][1], weight
    return tmp_edge

In [93]:
test_dic = make_nearestneighborhood_weights(node_dic, edge_dic, traffic_dic,1)
test_dic_dis = make_nearestneighborhood_weights(node_dic, edge_dic, traffic_dic, 0)


In [91]:
traffic_weightnn = {}
for node in neighbor_dic:
    traffic_weight = 0
    for no in neighbor_dic[node]:
        traffic_weight += traffic_dic[no] 
        weight = traffic_weight + traffic_dic[node]
    traffic_weightnn[node] = weight
    
#print traffic_weightnn
#for edge in edge_dic:
    #print accident_weight_btw_nodes(edge_dic[edge][0], edge_dic[edge][1], traffic_weightnn)

In [67]:
def make_weights(node_dict,edge_dict,feature_dict,feature_factor):
    tmp_dic = {}
    for feat in feature_dict:
        node = feature_dict[feat][3]
        if node != 'NULL':
            if node in tmp_dic:
                tmp_dic[node] =  tmp_dic[node] + 1
            else:
                tmp_dic[node] = feature_dict[feat][2]
                
        else:
            pass
    tmp_edge = {}
    for edge in edge_dict:
        #print edge_dict[edge][0], edge_dict[edge][1]
        acc_wei = accident_weight_btw_nodes(edge_dict[edge][0], edge_dict[edge][1], tmp_dic)
        weight = ((feature_factor*acc_wei) + 1)*distance_btw_nodes(edge_dict[edge][0],edge_dict[edge][1],node_dic)
        tmp_edge[edge] = edge_dict[edge][0], edge_dict[edge][1], weight
    return tmp_edge
#returns new edge function with weights 
#feature_factor = 0  ===no accident weightin
#feature_factor === weight you put on the increased distances associated with accidents


test_dist_weight = make_weights(node_dic, edge_dic, acc_dic,0)
test_dist_weight5 = make_weights(node_dic, edge_dic, acc_dic,10) # works

In [106]:
for edge in edge_dic:
    print test_dist_weight[edge][2], test_dic[edge][2], test_dic5[edge][2]

0.0486360837335 0.0486360837335 0.0486360837335
0.0334517129865 0.0411713390603 771.996059094
0.0616917293426 0.123383458685 6169.23462599
0.126810887533 0.253621775067 12681.2155642
0.0637233925936 0.106205654323 4248.2898963
0.0698894236525 0.116482372754 4659.36479959
0.0449640158678 0.0449640158678 0.0449640158678
0.00454723707914 0.00454723707914 0.00454723707914
0.052264797412 0.0895967955634 3733.25207994
0.00224890071417 0.00224890071417 0.00224890071417
0.0266981120352 0.0266981120352 0.0266981120352
0.0330479090014 0.0330479090014 0.0330479090014
0.0111305955454 0.0111305955454 0.0111305955454
0.0784738608722 0.0784738608722 0.0784738608722
0.0683808651185 0.0683808651185 0.0683808651185
0.0852893929722 0.170578785944 8529.02458661
0.114336392421 0.14972622817 3539.09791131
0.048418243053 0.0968364861061 4841.87272355
0.0496527330544 0.0496527330544 0.0496527330544
0.0173351742029 0.0173351742029 0.0173351742029
0.144650449879 0.201477412331 5682.84089569
0.011684676045 0.011

In [103]:
#cur.execute("""
#CREATE TABLE bk_weights_acc_nn(
#    edgeid BIGINT NOT NULL,
#    node1 BIGINT NOT NULL,
#    node2 BIGINT NOT NULL,
#    weight FLOAT(12,8) NOT NULL,
#    PRIMARY KEY (edgeid)
#    )
#    """)

add_edge= ("INSERT INTO bk_weights_acc_nn "
           " (edgeid, node1, node2, weight)"
           " VALUES (%s, %s,%s, %s)")

    
for ed in test_dic:
    edge_data = ed, test_dic[ed][0], test_dic[ed][1], test_dic[ed][2]
    cur.execute(add_edge, edge_data)
db_connect.commit()


IntegrityError: (1062, "Duplicate entry '5679816' for key 'PRIMARY'")

In [98]:
cur.close()
db_connect.close()

ProgrammingError: closing a closed connection

In [104]:
db_connect.commit()
